#Packages#

In [1]:
!pip install linear-tree

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd
import sklearn, numpy, joblib, scipy.stats

from sklearn.metrics import classification_report
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Data Import/Inspection/Cleaning#

In [3]:
df = pd.read_csv("/content/drive/MyDrive/HIDS 509/Project/Extracted Radiomic Features.csv")
df.head()

,Unnamed: 0,diagnostics_Versions_PyRadiomics,diagnostics_Versions_Numpy,diagnostics_Versions_SimpleITK,diagnostics_Versions_PyWavelet,diagnostics_Versions_Python,diagnostics_Configuration_Settings,diagnostics_Configuration_EnabledImageTypes,diagnostics_Image-original_Hash,diagnostics_Image-original_Dimensionality,...,original_glszm_SmallAreaHighGrayLevelEmphasis,original_glszm_SmallAreaLowGrayLevelEmphasis,original_glszm_ZoneEntropy,original_glszm_ZonePercentage,original_glszm_ZoneVariance,original_ngtdm_Busyness,original_ngtdm_Coarseness,original_ngtdm_Complexity,original_ngtdm_Contrast,original_ngtdm_Strength
0,900-00-5380,v3.0.1,1.22.4,2.2.1,1.4.1,3.9.16,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},5cbffddf539693c834a2ce0b9a93ca12ef590c4a,3D,...,157.022700,0.018078,6.993922,0.063631,3667.984726,0.521993,0.003392,478.947430,0.049892,1.599278
1,900-00-5381,v3.0.1,1.22.4,2.2.1,1.4.1,3.9.16,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},3640780e00c05893ff7f6f8bceb8e327fa313ecf,3D,...,18.096301,0.028771,5.195730,0.028809,78167.273097,10.545938,0.001076,27.300229,0.010473,0.093957
2,900-00-5385,v3.0.1,1.22.4,2.2.1,1.4.1,3.9.16,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},5523b17154aceb63840c645d12b812334fe33034,3D,...,10.173108,0.036043,4.194625,0.007755,279195.361570,9.176325,0.002123,5.661678,0.004584,0.072928
3,900-00-5396,v3.0.1,1.22.4,2.2.1,1.4.1,3.9.16,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},6491f16df057d97600c88130153b1d42719758c7,3D,...,28.720473,0.020997,6.189353,0.012516,296225.015122,10.642976,0.000642,45.215343,0.012831,0.088051
4,900-00-5404,v3.0.1,1.22.4,2.2.1,1.4.1,3.9.16,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},550d432eff845a7f06c84fcb28375d516be50b47,3D,...,4.755333,0.098691,4.243839,0.047228,12554.230624,15.572621,0.004650,3.256485,0.014443,0.047426


In [4]:
scaler = joblib.load("/content/drive/MyDrive/HIDS 509/Project/Models/scaler.bin")
cols = scaler.get_feature_names_out()
X = df.drop(columns = [col for col in df if col not in cols])
X = df[cols]
id = df["Unnamed: 0"]
X.columns

Index(['original_glrlm_GrayLevelNonUniformity',
       'original_glrlm_HighGrayLevelRunEmphasis',
       'original_glrlm_LongRunHighGrayLevelEmphasis',
       'original_glrlm_RunEntropy',
       'original_glrlm_ShortRunLowGrayLevelEmphasis',
       'original_glszm_GrayLevelNonUniformity', 'original_ngtdm_Busyness',
       'original_shape_MeshVolume', 'original_shape_SurfaceVolumeRatio',
       'original_shape_VoxelVolume'],
      dtype='object')

In [5]:
X.replace([numpy.inf, -numpy.inf], numpy.nan, inplace = True)
X.columns[X.isnull().any()]

<ipython-input-5-086461b570cc>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.replace([numpy.inf, -numpy.inf], numpy.nan, inplace = True)


Index([], dtype='object')

In [6]:
X = scaler.transform(X)

#Load Models & Predict#

In [7]:
le = sklearn.preprocessing.LabelEncoder().fit(["GBM", "Astrocytoma", "Oligodendroglioma"])

In [8]:
SVM = joblib.load("/content/drive/MyDrive/HIDS 509/Project/Models/SVM.sav")
SVM_y_pred = le.inverse_transform(SVM.predict(X))

RFC = joblib.load("/content/drive/MyDrive/HIDS 509/Project/Models/RFC.sav")
RFC_y_pred = le.inverse_transform(RFC.predict(X))

BC = joblib.load("/content/drive/MyDrive/HIDS 509/Project/Models/BC.sav")
BC_y_pred = le.inverse_transform(BC.predict(X))

ABC = joblib.load("/content/drive/MyDrive/HIDS 509/Project/Models/ABC.sav")
ABC_y_pred = le.inverse_transform(ABC.predict(X))

kmeans = joblib.load("/content/drive/MyDrive/HIDS 509/Project/Models/KMeans.sav")
kmeans_y_pred = le.inverse_transform(kmeans.predict(X))

LT = joblib.load("/content/drive/MyDrive/HIDS 509/Project/Models/LT.sav")
LT_y_pred = le.inverse_transform(LT.predict(X))

stacked = joblib.load("/content/drive/MyDrive/HIDS 509/Project/Models/stacked.sav")
stacked_y_pred = le.inverse_transform(stacked.predict(X))


X_cluster = joblib.load("/content/drive/MyDrive/HIDS 509/Project/Models/cluster.sav").transform(X)
wrapped = joblib.load("/content/drive/MyDrive/HIDS 509/Project/Models/wrapped.sav")
wrapped_y_pred = le.inverse_transform(wrapped.predict(X_cluster))

In [9]:
data = {"ID": id,"SVM": SVM_y_pred, "Random Forest": RFC_y_pred, "Bagging": BC_y_pred, "AdaBoost": ABC_y_pred, 
        "K-Means": kmeans_y_pred, "LT": LT_y_pred, "stacked": stacked_y_pred, "wrapped": wrapped_y_pred}
results = pd.DataFrame(data)
results.head()

,ID,SVM,Random Forest,Bagging,AdaBoost,K-Means,LT,stacked,wrapped
0,900-00-5380,Oligodendroglioma,Astrocytoma,Astrocytoma,Astrocytoma,GBM,Astrocytoma,Astrocytoma,Oligodendroglioma
1,900-00-5381,Oligodendroglioma,Oligodendroglioma,Oligodendroglioma,Astrocytoma,GBM,GBM,Oligodendroglioma,Oligodendroglioma
2,900-00-5385,Oligodendroglioma,Oligodendroglioma,Oligodendroglioma,Astrocytoma,GBM,Oligodendroglioma,Oligodendroglioma,Oligodendroglioma
3,900-00-5396,Oligodendroglioma,Oligodendroglioma,Oligodendroglioma,Astrocytoma,GBM,Astrocytoma,Oligodendroglioma,Oligodendroglioma
4,900-00-5404,Oligodendroglioma,Oligodendroglioma,Astrocytoma,Astrocytoma,GBM,GBM,GBM,Astrocytoma


#Performance Comparison#

In [10]:
ground_truth = pd.read_csv("/content/drive/MyDrive/HIDS 509/Project/Rembrandt_Clinical_Truth_Final64Set.tsv", sep = '\t')
ground_truth

,64.patient.set,AGE_RANGE,GENDER,DISEASE_TYPE
0,900_00_5299,NaN,NaN,NaN
1,900_00_5303,NaN,NaN,NaN
2,900_00_5308,NaN,,ASTROCYTOMA
3,900_00_5316,NaN,,GBM
4,900_00_5317,NaN,,GBM
...,...,...,...,...
59,HF1606,60-64,MALE,ASTROCYTOMA
60,HF1613,35-39,MALE,ASTROCYTOMA
61,HF1652,NaN,NaN,NaN
62,HF1677,50-54,MALE,ASTROCYTOMA


In [11]:
results = results.merge(ground_truth, left_on = "ID", right_on = "64.patient.set")
results.columns

Index(['ID', 'SVM', 'Random Forest', 'Bagging', 'AdaBoost', 'K-Means', 'LT',
       'stacked', 'wrapped', '64.patient.set', 'AGE_RANGE', 'GENDER',
       'DISEASE_TYPE'],
      dtype='object')

In [12]:
results = results.drop(["64.patient.set", "AGE_RANGE", "GENDER"], axis = 1)
results = results.dropna(axis = 0)
results.head()

,ID,SVM,Random Forest,Bagging,AdaBoost,K-Means,LT,stacked,wrapped,DISEASE_TYPE
0,HF1156,Oligodendroglioma,Oligodendroglioma,Oligodendroglioma,Astrocytoma,Oligodendroglioma,Oligodendroglioma,Oligodendroglioma,Oligodendroglioma,ASTROCYTOMA
1,HF1167,Oligodendroglioma,Oligodendroglioma,Oligodendroglioma,Oligodendroglioma,GBM,GBM,GBM,GBM,OLIGODENDROGLIOMA
2,HF1185,Oligodendroglioma,Astrocytoma,Astrocytoma,GBM,GBM,GBM,Astrocytoma,Oligodendroglioma,ASTROCYTOMA
3,HF1191,Oligodendroglioma,GBM,Oligodendroglioma,GBM,GBM,GBM,Oligodendroglioma,Oligodendroglioma,GBM
4,HF1219,Oligodendroglioma,Astrocytoma,GBM,GBM,GBM,Astrocytoma,Astrocytoma,Oligodendroglioma,OLIGODENDROGLIOMA


In [13]:
results.loc[results["DISEASE_TYPE"] == "ASTROCYTOMA",  "DISEASE_TYPE"] = "Astrocytoma"
results.loc[results["DISEASE_TYPE"] == "OLIGODENDROGLIOMA",  "DISEASE_TYPE"] = "Oligodendroglioma"
results.to_csv("/content/drive/MyDrive/HIDS 509/Project/Predicted Results.csv")

In [14]:
print(classification_report(le.transform(results["DISEASE_TYPE"]), le.transform(results["SVM"]), zero_division = 0))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        24
           1       0.00      0.00      0.00         8
           2       0.18      0.88      0.30         8

    accuracy                           0.17        40
   macro avg       0.06      0.29      0.10        40
weighted avg       0.04      0.17      0.06        40



In [15]:
print(classification_report(le.transform(results["DISEASE_TYPE"]), le.transform(results["Random Forest"]), zero_division = 0))

              precision    recall  f1-score   support

           0       0.62      0.21      0.31        24
           1       0.40      0.50      0.44         8
           2       0.18      0.50      0.27         8

    accuracy                           0.33        40
   macro avg       0.40      0.40      0.34        40
weighted avg       0.49      0.33      0.33        40



In [16]:
print(classification_report(le.transform(results["DISEASE_TYPE"]), le.transform(results["Bagging"]), zero_division = 0))

              precision    recall  f1-score   support

           0       0.86      0.25      0.39        24
           1       0.33      0.38      0.35         8
           2       0.21      0.62      0.31         8

    accuracy                           0.35        40
   macro avg       0.47      0.42      0.35        40
weighted avg       0.62      0.35      0.37        40



In [17]:
print(classification_report(le.transform(results["DISEASE_TYPE"]), le.transform(results["AdaBoost"]), zero_division = 0))

              precision    recall  f1-score   support

           0       0.68      0.71      0.69        24
           1       0.33      0.50      0.40         8
           2       0.33      0.12      0.18         8

    accuracy                           0.55        40
   macro avg       0.45      0.44      0.43        40
weighted avg       0.54      0.55      0.53        40



In [18]:
print(classification_report(le.transform(results["DISEASE_TYPE"]), le.transform(results["K-Means"]), zero_division = 0))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        24
           1       0.24      1.00      0.38         8
           2       0.33      0.25      0.29         8

    accuracy                           0.25        40
   macro avg       0.19      0.42      0.22        40
weighted avg       0.11      0.25      0.13        40



In [19]:
print(classification_report(le.transform(results["DISEASE_TYPE"]), le.transform(results["LT"]), zero_division = 0))

              precision    recall  f1-score   support

           0       0.57      0.17      0.26        24
           1       0.32      0.88      0.47         8
           2       0.18      0.25      0.21         8

    accuracy                           0.33        40
   macro avg       0.36      0.43      0.31        40
weighted avg       0.44      0.33      0.29        40



In [20]:
print(classification_report(le.transform(results["DISEASE_TYPE"]), le.transform(results["stacked"]), zero_division = 0))

              precision    recall  f1-score   support

           0       0.67      0.17      0.27        24
           1       0.33      0.38      0.35         8
           2       0.16      0.50      0.24         8

    accuracy                           0.28        40
   macro avg       0.39      0.35      0.29        40
weighted avg       0.50      0.28      0.28        40



In [21]:
print(classification_report(le.transform(results["DISEASE_TYPE"]), le.transform(results["wrapped"]), zero_division = 0))

              precision    recall  f1-score   support

           0       0.25      0.04      0.07        24
           1       0.22      0.25      0.24         8
           2       0.11      0.38      0.17         8

    accuracy                           0.15        40
   macro avg       0.19      0.22      0.16        40
weighted avg       0.22      0.15      0.12        40

